In [2]:
#credits to https://www.pythonforfinance.net/2019/07/02/investment-portfolio-optimisation-with-python-revisited/

import pandas as pd  
import numpy as np
import pandas_datareader.data as web
import datetime as dt
from datetime import date, datetime, timedelta

import scipy.optimize as sco
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [161]:
#credits to https://www.pythonforfinance.net/2019/07/02/investment-portfolio-optimisation-with-python-revisited/


def calc_portfolio_perf(weights, mean_returns, cov, rf):
    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    return portfolio_return, portfolio_std, sharpe_ratio

def simulate_random_portfolios(num_portfolios, mean_returns, cov, rf):
    results_matrix = np.zeros((len(mean_returns)+3, num_portfolios))
    for i in range(num_portfolios):
        weights = np.random.random(len(mean_returns))
        weights /= np.sum(weights)
        portfolio_return, portfolio_std, sharpe_ratio = calc_portfolio_perf(weights, mean_returns, cov, rf)
        results_matrix[0,i] = portfolio_return
        results_matrix[1,i] = portfolio_std
        results_matrix[2,i] = sharpe_ratio
        #iterate through the weight vector and add data to results array
        for j in range(len(weights)):
            results_matrix[j+3,i] = weights[j]
            
    results_df = pd.DataFrame(results_matrix.T,columns=['ret','stdev','sharpe'] + [ticker for ticker in tickers])
        
    return results_df


def benchmark(indices):
    indices = indices
    start = date.today() - timedelta(days = 3650*2)
    end = date.today()

    starter_data = {'benchmark':'0','ret':'0', 'stdev':'0', 'sharpe':'0'}
    df = pd.DataFrame(starter_data, index = starter_data.keys())[0:1].reset_index(drop = True)

    for index in indices:

        series = web.DataReader(index, 'yahoo', start, end)['Adj Close']
        ret = series.pct_change().mean()*252
        stdev = series.pct_change().std()*np.sqrt(252)
        sharpe = (ret-rf)/stdev
        benchmark_data = {'benchmark':index,'ret':ret, 'stdev':stdev, 'sharpe':sharpe}
        new_row = pd.DataFrame(benchmark_data, index = benchmark_data.keys())[0:1].reset_index(drop = True)
        df = df.append(new_row)
    df = df.reset_index(drop = True).drop(0)
    return(df)

def capital_allocation(NAV, input_results, tickers):
    pct_allocation = {}
    cap_allocation = {}
    for ticker in tickers:
        pct_allocation[ticker] = float(input_results[ticker])
        cap_allocation[ticker] = round(pct_allocation[ticker]*NAV,2)
    return(cap_allocation)

In [189]:
tickers = ['TLT','SPY','GLD','LQD']
start = date.today() - timedelta(days = 3650*2)
end = date.today()
df = pd.DataFrame([web.DataReader(ticker, 'yahoo', start, end)['Adj Close'] for ticker in tickers]).T
df.columns = tickers

mean_returns = df.pct_change().mean()
cov = df.pct_change().cov()
num_portfolios = 10000
rf = 0.0
results_frame = simulate_random_portfolios(num_portfolios, mean_returns, cov, rf)

results_frame[results_frame['sharpe']==results_frame['sharpe'].max()]

,ret,stdev,sharpe,TLT,SPY,GLD,LQD
3265,0.080288,0.075195,1.067738,0.314997,0.210204,0.1679,0.306899


In [190]:
results_frame['levered_ret']=[0.0]*len(results_frame)
results_frame['levered_stdev']=[0.0]*len(results_frame)
results_frame['levered_sharpe'] = [0.0]*len(results_frame)

leverage = 1.5
margin_int =0.015

for i in range(len(results_frame)):
    results_frame['levered_ret'][i] = results_frame['ret'][i] * leverage - (leverage - 1)*margin_int
    results_frame['levered_stdev'][i] = results_frame['stdev'][i] * leverage
    results_frame['levered_sharpe'][i] = results_frame['levered_ret'][i]/results_frame['levered_stdev'][i]

max_levered_sharpe = results_frame['levered_sharpe'].max()
max_sharpe = results_frame['sharpe'].max()
results_frame[results_frame['levered_sharpe']==max_levered_sharpe]

,ret,stdev,sharpe,TLT,SPY,GLD,LQD,levered_ret,levered_stdev,levered_sharpe
3265,0.080288,0.075195,1.067738,0.314997,0.210204,0.1679,0.306899,0.112932,0.112792,1.001243


In [192]:
benchmark(['SPY','IWM','QQQ','VT'])

,benchmark,ret,stdev,sharpe
1,SPY,0.0779616,0.197848,0.394048
2,IWM,0.0954673,0.242404,0.393836
3,QQQ,0.0921604,0.264479,0.34846
4,VT,0.0850438,0.22475,0.378392


In [233]:
tickers = ['TLT','SPY','GLD','LQD']
input_results = results_frame[results_frame['levered_sharpe']==max_levered_sharpe]
NAV = 50000

def capital_allocation(NAV, input_results, tickers):
    pct_allocation = {}
    cap_allocation = {}
    for ticker in tickers:
        pct_allocation[ticker] = float(input_results[ticker])
        cap_allocation[ticker] = round(pct_allocation[ticker]*NAV,2)
    return(cap_allocation)



In [ ]:
def capital_allocation(NAV, holdings, input_results):
    for t